In [62]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

In [63]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [64]:
model=TheModelClass()
solver=optim.SGD(model.parameters(),lr=1e-3,momentum=0.9,weight_decay=0.9)

In [65]:
#show state_dict
for vname,vvars in model.state_dict().items():
        print(vname,':',vvars.size())

conv1.weight : torch.Size([6, 3, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])


In [66]:
for vname,vvars in solver.state_dict().items():
        print(vname,':',vvars)

state : {}
param_groups : [{'nesterov': False, 'dampening': 0, 'lr': 0.001, 'params': [140333420703464, 140333420703104, 140333420702672, 140333420701016, 140333420701232, 140333420700728, 140333420700512, 140333420703608, 140333420748016, 140333420746792], 'weight_decay': 0.9, 'momentum': 0.9}]


In [72]:
np.random.seed(10)
X=torch.Tensor(np.random.randn(5,3,32,32))
model.forward(X)

tensor([[ 0.0956,  0.1622,  0.0561,  0.1320, -0.0684,  0.0030, -0.0197, -0.0924,
          0.0181, -0.0959],
        [ 0.0828,  0.1817,  0.0557,  0.1211, -0.0850,  0.0026, -0.0338, -0.1032,
          0.0109, -0.1165],
        [ 0.1125,  0.1723,  0.0262,  0.1144, -0.0741,  0.0027,  0.0074, -0.1032,
          0.0069, -0.0858],
        [ 0.0641,  0.1601,  0.0607,  0.1220, -0.0796,  0.0024, -0.0532, -0.1067,
          0.0293, -0.1130],
        [ 0.0879,  0.1398,  0.0615,  0.1218, -0.1160, -0.0021, -0.0351, -0.0939,
          0.0303, -0.1369]], grad_fn=<AddmmBackward>)

In [ ]:
#save model
PATH='weights/classifier.pt'
torch.save(model.state_dict(),PATH)

In [73]:
#reload model
PATH='weights/classifier.pt'
model.load_state_dict(torch.load(PATH))
# model.eval() to set dropout and batch normalization layers to 
# evaluation mode before running inference. 
# Failing to do this will yield inconsistent inference results.
model.eval()
#then run above code, see follow result!!

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

tensor([[ 0.0956,  0.1622,  0.0561,  0.1320, -0.0684,  0.0030, -0.0197, -0.0924,
          0.0181, -0.0959],
        [ 0.0828,  0.1817,  0.0557,  0.1211, -0.0850,  0.0026, -0.0338, -0.1032,
          0.0109, -0.1165],
        [ 0.1125,  0.1723,  0.0262,  0.1144, -0.0741,  0.0027,  0.0074, -0.1032,
          0.0069, -0.0858],
        [ 0.0641,  0.1601,  0.0607,  0.1220, -0.0796,  0.0024, -0.0532, -0.1067,
          0.0293, -0.1130],
        [ 0.0879,  0.1398,  0.0615,  0.1218, -0.1160, -0.0021, -0.0351, -0.0939,
          0.0303, -0.1369]], grad_fn=<AddmmBackward>)

In [77]:
#now let save a general model
model=TheModelClass()
lr=torch.Tensor([0.5])
lr.requires_grad
solver=optim.Adam(model.parameters(),lr)

In [85]:
def save(model,solver,PATH):
    state={'model':model.state_dict(),
          'solver':solver.state_dict()}
    torch.save(state,PATH)
PATH='weights/general_model.tar'
save(model,solver,PATH)

In [84]:
def load(model,solver,PATH):
    state=torch.load(PATH)
    model.load_state_dict(state['model'])
    solver.load_state_dict(state['solver'])

In [90]:
model=TheModelClass()
solver=optim.Adam(model.parameters(),0.9)
print(solver)
PATH='weights/general_model.tar'
load(model,solver,PATH)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.9
    weight_decay: 0
)


In [114]:
#loose load,strict=False will allow mismatch error
class AnotherModel(TheModelClass):
    def __init__(self):
        super().__init__() 
        #foo layer
        self.foo=nn.Linear(22,22)
        self.xx=torch.rand((3,2),requires_grad=True)

In [115]:
model=AnotherModel()
PATH='weights/general_model.tar'
model.load_state_dict(torch.load(PATH)['model'],strict=False)

Save on GPU, Load on CPU

In [127]:
device=torch.device('cpu')
state_dict=torch.load('/home/zhangxk/AIProject/CCPD/weight/fh02.pth',map_location=device)
for name in state_dict:
    print(name)

module.wR2.module.features.0.0.weight
module.wR2.module.features.0.0.bias
module.wR2.module.features.0.1.weight
module.wR2.module.features.0.1.bias
module.wR2.module.features.0.1.running_mean
module.wR2.module.features.0.1.running_var
module.wR2.module.features.1.0.weight
module.wR2.module.features.1.0.bias
module.wR2.module.features.1.1.weight
module.wR2.module.features.1.1.bias
module.wR2.module.features.1.1.running_mean
module.wR2.module.features.1.1.running_var
module.wR2.module.features.2.0.weight
module.wR2.module.features.2.0.bias
module.wR2.module.features.2.1.weight
module.wR2.module.features.2.1.bias
module.wR2.module.features.2.1.running_mean
module.wR2.module.features.2.1.running_var
module.wR2.module.features.3.0.weight
module.wR2.module.features.3.0.bias
module.wR2.module.features.3.1.weight
module.wR2.module.features.3.1.bias
module.wR2.module.features.3.1.running_mean
module.wR2.module.features.3.1.running_var
module.wR2.module.features.4.0.weight
module.wR2.module.feat

In [132]:
a='module.wR2.module.features.0.0.weight'

In [137]:
if a.startswith('module.wR2'):
    print(a[len('modele.'):])

wR2.module.features.0.0.weight


In [ ]:
a.replace('.module','')